This is a vision application of the AIRSAS Image Classification study for my thesis. Inputs are in the form of 875 by 656 pixel beamformed images from the AIRSAS. Outputs are the predicted letter present in the scan. The training set consists of imagery of 5,000 numbers generated from MATLAB. In this implementation, several key features will be determined from the generated input imagery to make predictions on AIRSAS imagery.

NOTE: This notebook makes use of the fastai library. Run the code cell below to get the necessary files. 

In [ ]:
!pip install fastai==2.1.10

Next, we will need to import all the necessary libraries for this project.

In [ ]:
from fastai.vision.all import *
import os
import numpy as np
import random
import pandas as pd
import time
from pathlib import Path
import zipfile as zf
from IPython.display import Image
from PIL import Image, ImageOps

Next, we will allocate the testing parameters for this implementation. Epochs has been set to 100, so the learners will run for 100 epochs. The layer setup has been set to the Fast.AI default. n_classes contains the individual numbers that will be tested.

In [ ]:
epochs = 10
#Full n_classes = ['1','2','3','4','5','6','7','8','9','0']
n_classes =  ['1','2','3','4','5','6','7','8','9','0']
print('Training_' + '_'.join([str(elem) for elem in n_classes]))

Finally, we will allocate the constants for this implementation. metrics specifies the values our learner model will display per epoch.

In [ ]:
metrics = [accuracy,error_rate]

In the next two cells, we will set up the folders needed to organize our input data. Even if the folders are already setup, the first cell must be run in order to allocate the paths used later on.

In [ ]:
AIRSAS_Classifier_Data = Path('AIRSAS_Classifier_Data_flipped')
Test_Imagery = Path(str(AIRSAS_Classifier_Data) + '/Test_Imagery')
Train_Imagery = Path(str(AIRSAS_Classifier_Data) + '/Train_Imagery')

This cell makes the directories if they don't exist already.

In [ ]:
AIRSAS_Classifier_Data.mkdir(parents=True, exist_ok=True)
Test_Imagery.mkdir(parents=True, exist_ok=True)
Train_Imagery.mkdir(parents=True, exist_ok=True)

We now have to allocate training data.  Zip all the test image files in a file titled "Train_numbers.zip".
Run this cell to extract all the files.

In [ ]:
files = zf.ZipFile(str(AIRSAS_Classifier_Data) + '/Train_Imagery/Train_numbers.zip', 'r')
files.extractall(str(Train_Imagery))
files.close()
os.remove(str(Train_Imagery) + '/Train_numbers.zip')

I realized that the Matlab script for generated image data was making images that were mirror flipped vertically. Run the next cell to fix this issue.

In [ ]:
os.rmdir(str(Train_Imagery) + '/.ipynb_checkpoints')
for file in os.listdir(Train_Imagery):
  filename = os.fsdecode(file)
  im = Image.open(str(Train_Imagery) + '/' + filename)
  im_flip = ImageOps.flip(im)
  im_flip.save(str(Train_Imagery) + '/' + filename)
  print(filename)

Next step is to allocate the testing data. Zip all of the output images produced by the beamformer script into an archive titled "Test_numbers.zip". Run this cell to extract all the files.

In [ ]:
files = zf.ZipFile(str(AIRSAS_Classifier_Data) + "/Test_Imagery/AIRSAS Scans.zip", 'r')
files.extractall(str(Test_Imagery))
files.close()
os.remove(str(Test_Imagery) + '/AIRSAS Scans.zip')

Data Collection on the AIRSAS results in mirrored scans. Run this cell to fix that. 

In [ ]:
os.rmdir(str(Train_Imagery) + '/.ipynb_checkpoints')
PFA = Path(str(Test_Imagery) + '/PFA')
for file in os.listdir(PFA):
  filename = os.fsdecode(file)
  im = Image.open(str(PFA) + '/' + filename)
  im_flip = ImageOps.mirror(im)
  im_flip.save(str(PFA) + '/' + filename)
  print(filename)

To verify that things are functioning, the next cell will choose a random image from the train set.

In [ ]:
random_image = random.choice(os.listdir(str(Train_Imagery)))
random_image_path = Path(str(Train_Imagery) + '/' + str(random_image))
print('Random image is: ' + str(random_image))

The random image is displayed.

In [ ]:
display(Image.open(random_image_path))

The individual classes or genres to be tested.

In [ ]:
classes = n_classes
classes

This cell allocates the data into a format used by the Fast.ai

In [ ]:
files = get_image_files(Train_Imagery)
def label_func(x): return x.name[6]

dls = ImageDataLoaders.from_path_func(Train_Imagery, files, label_func, batch_size = 10, size = 28)

This cell shows an example batch of values in our dataset.

In [ ]:
dls.show_batch()

This cell allocates our learner model with the layers and metrics we allocated earlier.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=metrics)

This cell runs the command lr_find(), which will determine some examples for a good learning rate by displaying the learning rate that yeild minimum loss or steepest descent. Either use the default of 3e-3, choose a rate between the two values displayed, or plug the two values into the mean formula below.

In [ ]:
learn.lr_find()

This cell finds the mean between the two learning rate values.

In [ ]:
lr_min=0.003981071710586548
lr_steep=6.309573450380412e-07
lr = (lr_min + lr_steep)/2 

This cell fits our learner to the training set for our specified number of epochs. 

In [ ]:
learn.fit_one_cycle(epochs)

The next cells will export our model for later use. 

In [ ]:
learn.save('stage-' + str(epochs) + 'epochs')

In [ ]:
learn.export(fname = 'model-flipped-' + str(epochs) + 'epochs')

This is where we can load a previous model or stage if desired

In [ ]:
learn.load('stage-100epochs-1')
#learn = load_learner(str(Train_Imagery) + '/model-flipped-3-10epoch.pkl')

The next cell will show predicted results for our model.

In [ ]:
learn.show_results()

In order to verify performance, we will re run the predictions on our training set. 

In [ ]:
correct = 0
incorrect = 0

for i in range(0,len(files)):
  if learn.predict(files[i])[0] == str(files[i])[-16]:
    correct = correct + 1
  else:
    incorrect = incorrect + 1
print(correct)
print(incorrect)

Next, we will plot the confusion matrix.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

Finally, we will make predictions on our test data sets.
This next cell will loop through the test set and compare the prediction scores for each number with the actual quantities of each. Finally, the accuracy will be computed.

In [ ]:
PFA = Path(str(Test_Imagery) + '/PFA')
testfiles = get_image_files(PFA)

for i in range(0,len(testfiles)):
  number = str(testfiles[i])[-5]
  prediction = learn.predict(testfiles[i])
  print(number + '_' + str(prediction)[2])